In [1]:
import nltk 
import pandas as pd 
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%pylab inline
from stop_words import get_stop_words
from nltk.corpus import stopwords
import pycountry as pc

Populating the interactive namespace from numpy and matplotlib


In [2]:
email_path = "./hillary-clinton-emails/Emails.csv"
email_df = pd.read_csv(email_path, encoding="utf-8")

In [3]:
email_df.columns

Index([u'Id', u'DocNumber', u'MetadataSubject', u'MetadataTo', u'MetadataFrom',
       u'SenderPersonId', u'MetadataDateSent', u'MetadataDateReleased',
       u'MetadataPdfLink', u'MetadataCaseNumber', u'MetadataDocumentClass',
       u'ExtractedSubject', u'ExtractedTo', u'ExtractedFrom', u'ExtractedCc',
       u'ExtractedDateSent', u'ExtractedCaseNumber', u'ExtractedDocNumber',
       u'ExtractedDateReleased', u'ExtractedReleaseInPartOrFull',
       u'ExtractedBodyText', u'RawText'],
      dtype='object')

In [4]:
email_df.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


# QUESTION 1 

## WORLDCLOUD RAW

In [5]:
email_df.shape

(7945, 22)

In [6]:
email_df.dropna(subset=['ExtractedBodyText'], inplace=True)

In [7]:
email_df.shape

(6742, 22)

In [8]:
text = email_df['ExtractedBodyText']

In [9]:
text.head()

1    B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...
2                                                  Thx
4    H <hrod17@clintonemail.com>\nFriday, March 11,...
5    Pis print.\n-•-...-^\nH < hrod17@clintonernail...
7    H <hrod17@clintonemail.corn>\nFriday, March 11...
Name: ExtractedBodyText, dtype: object

In [10]:
test_text = " ".join(text) # to add space

wc = WordCloud(background_color="white", max_words=2000, mask=alice_mask,stopwords=stopwords)

In [ ]:
# Generate a word cloud image
wordcloud = WordCloud().generate(test_text)
# Display the generated image:
# the matplotlib way: 
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")


##  PREPROCESSING

### TOKENIZATION

For example, you can define a tokenizer that picks out sequences of alphanumeric characters as tokens and drops everything else:



the following tokenizer forms tokens out of alphabetic sequences,
money expressions,

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+|\$')

In [ ]:
tokens = tokenizer.tokenize(test_text)

In [ ]:
tokens

### CASE FOLDING

In [ ]:
tokens_lower_case = [token.lower() for token in tokens ]

In [ ]:
tokens_lower_case

### REMOVE STOPWORDS 

In [ ]:
stop_words = get_stop_words('english')

In [ ]:
len(tokens_lower_case)

In [ ]:
filtered_words_tmp = [word for word in tokens_lower_case if word not in stop_words]

In [ ]:
len(filtered_words_tmp)

In [ ]:
filtered_words_lower_case = [word for word in filtered_words_tmp if word not in stopwords.words('english')]

In [ ]:
len(filtered_words_lower_case)

### STEMMING

In [ ]:
porter = nltk.PorterStemmer()

token_stem_lower_case = [porter.stem(token) for token in filtered_words_lower_case]

In [ ]:
token_stem_lower_case

## WORLDCLOUD NEW 

In [ ]:
new_text = " ".join(token_stem_lower_case)

In [ ]:
# Generate a word cloud image
wordcloud = WordCloud().generate(new_text)
# Display the generated image:
# the matplotlib way: 
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")


# QUESTION  2

 1 - check if there is a country and also the abreviation in bodytext one by one
 2 - If is not there ignore the email
 3 - If it's there add the country in the new column in a list (it may be multiple)
 

In [ ]:
text = email_df['ExtractedBodyText']

In [25]:
from nltk.sentiment.util import *

/home/trinh/anaconda2/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [26]:
from nltk.sentiment import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

In [23]:
def get_polarity_score(text):
    """ return score between 0 and 1, 0 is bad 1 is happy"""
    vader_analyzer = SentimentIntensityAnalyzer()
    return (vader_analyzer.polarity_scores(text)['compound']+1)/2

In [71]:
def return_country(cont_list):
    tmp = []
    count = 0
    for token in cont_list:
        try:
            count=1
            tmp.append(pc.countries.get(name = token).name)
            
        except:
            None    
        try:
            if count !=1:
                tmp.append(pc.countries.get(alpha_2 = token).name)
        except:
            None
    return pd.Series(tmp)

In [13]:
email_df['token'] = email_df['ExtractedBodyText'].apply(nltk.word_tokenize)

In [14]:
email_df['token']

1       [B6, Thursday, ,, March, 3, ,, 2011, 9:45, PM,...
2                                                   [Thx]
4       [H, <, hrod17, @, clintonemail.com, >, Friday,...
5       [Pis, print, ., -•-, ..., -^, H, <, hrod17, @,...
7       [H, <, hrod17, @, clintonemail.corn, >, Friday...
8                                                   [FYI]
9       [B6, Wednesday, ,, September, 12, ,, 2012, 6:1...
10                                        [Fyi, B6, —, —]
11      [B6, Wednesday, ,, September, 12, ,, 2012, 6:1...
12                                                  [Fyi]
13      [Anne-Marie, Slaughter, Sunday, ,, March, 13, ...
14      [_, ..., .., From, Randolph, ,, Lawrence, M, S...
15      [I, asked, to, attend, your, svtc, today, with...
16        [Hope, ., See, picture, below, Kamala, sent, .]
17                                    [Another, photo, .]
18                                    [This, is, nice, .]
19      [Amazing, ., Sullivan, ,, Jacob, J, <, Sulliva...
20      [H, <,

In [19]:
email_df['country'] = email_df['token'].apply(return_country)

In [20]:
email_df['country'].head()

1    Saint Pierre and Miquelon
2                          NaN
4    Saint Pierre and Miquelon
5    Saint Pierre and Miquelon
7    Saint Pierre and Miquelon
Name: country, dtype: object

In [21]:
email_df.dropna(subset=['country'],inplace=True)

In [27]:
email_df['sentiment'] = email_df['ExtractedBodyText'].apply(get_polarity_score)

In [59]:
email_df.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText,country,token,sentiment
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...,Saint Pierre and Miquelon,"[B6, Thursday, ,, March, 3, ,, 2011, 9:45, PM,...",0.5000
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,Saint Pierre and Miquelon,"[H, <, hrod17, @, clintonemail.com, >, Friday,...",0.5000
5,6,C05739559,MEET THE RIGHT-WING EXTREMIST BEHIND ANTI-MUSL...,Russorv@state.gov,H,80.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739559...,F-2015-04841,...,"Wednesday, September 12, 2012 01:00 PM",F-2015-04841,C05739559,05/13/2015,RELEASE IN PART,Pis print.\n-•-...-^\nH < hrod17@clintonernail...,B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,Saint Pierre and Miquelon,"[Pis, print, ., -•-, ..., -^, H, <, hrod17, @,...",0.0799
7,8,C05739561,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739561...,F-2015-04841,...,NaN,F-2015-04841,C05739561,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.corn>\nFriday, March 11...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,Saint Pierre and Miquelon,"[H, <, hrod17, @, clintonemail.corn, >, Friday...",0.5000
9,10,C05739563,MORE ON LIBYA,H,NaN,NaN,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739563...,F-2015-04841,...,NaN,F-2015-04841,C05739563,05/13/2015,RELEASE IN PART,"B6\nWednesday, September 12, 2012 6:16 PM\nFwd...",UNCLASSIFIED\nU.S. Department of State\nCase N...,Saint Pierre and Miquelon,"[B6, Wednesday, ,, September, 12, ,, 2012, 6:1...",0.5000


In [65]:
email_df['ExtractedBodyText'][46]

u'http://www.salon.comi2o12/1o/or/gops_october_surprisei\nMonday, Oct 1, 2012 09:30 AM EDT\nGOP\'s October surprise?\nThey\'re calling it the "Jimmy Carter Strategy": Exploit\nLibya attack to paint Obama as weak on terrorism\nBy cY\'aia?,_\n(Credit: AP/AI Behrman/Salort)\nAccording to a highly reliable source, as Mitt Romney and President Barack Obama prepare for the\nfirst presidential debate Wednesday night, top Republican operatives are primed to unleash a new\ntwo-pronged offensive that will attack Obama as weak on national security, and will be based, in part,\non inW !ia:en,y-.3iTeForrnation regarding the attacks in Libya that killed U.S. ambassador Chris\nStevens on September\nThe source, who has first-hand knowledge of private, high-level conversations in the Romney tamp\nthat took place in Washington, DC last week, said that at various times the GOP strategists referred to\ntheir new operation as the Jimmy Carter Strategy or the October Surprise.\nHe added that they planned to

In [66]:
email_df['country'].value_counts()

Saint Pierre and Miquelon                586
Armenia                                  506
India                                    118
United States                             65
Micronesia, Federated States of           53
Haiti                                     45
Tonga                                     43
Afghanistan                               39
Israel                                    38
China                                     34
Montserrat                                24
Pakistan                                  22
Belarus                                   20
Austria                                   19
Iraq                                      15
Tuvalu                                    15
Mexico                                    14
Honduras                                  14
Ireland                                   14
Libya                                     11
Palau                                      9
Turkey                                     9
Indonesia 

In [55]:
pc.countries.get(name = u'Libya')

Country(alpha_2=u'LY', alpha_3=u'LBY', name=u'Libya', numeric=u'434', official_name=u'Libya')

In [40]:
email_df.groupby('country').mean()['sentiment']

country
Afghanistan                              0.731492
Albania                                  0.680600
Angola                                   0.500000
Antarctica                               0.956750
Argentina                                0.727642
Armenia                                  0.659920
Australia                                0.923863
Austria                                  0.672932
Azerbaijan                               0.998300
Bahrain                                  0.696200
Bangladesh                               0.894475
Barbados                                 0.946100
Belarus                                  0.539945
Belgium                                  0.860650
Bermuda                                  0.512900
Brazil                                   0.709367
Brunei Darussalam                        0.582317
Cameroon                                 0.882683
Canada                                   0.609064
Chile                                    0